## 2020-09-30: Exploring Accuracy for Orthogonalization Algorithms

*Last Updated*: 2020-09-30

### Authors

* Kevin Chu (kevin@velexi.com)

### Overview

In this Jupyter notebook, we explore the loss of orthogonality of modified Gram-Schmidt (MGS) and Householder orthogonalization algorithms.

### Key Results

* __Loss of Orthogonality__. For "random matrices", both MGS and Householder orthogonalization produce $Q$ matrices where $Q' * Q - I$ is within a couple of orders of magnitude of machine precision.

    * $|Q' * Q - I|_2$ tends to be about an order of magnitude smaller for Householder orthogonalization than for MGS.
    
    * The standard deviation of $|Q' * Q - I|_2$ tends to be about several orders of magnitude smaller for Householder orthogonalization than for MGS.
    
* __Accuracy of Determinant__. For "random matrices", both MGS and Householder orthogonalization produce factorizations where the relative error of $\det(Q R)$ is close to machine precision.

    * The relative error of the $\det(Q R)$ is very comparable for both algorithms.
    
### User parameters

* `n`: matrix size
* `num_samples`: number of matrices to sample

In [1]:
# --- User parameters

n = 100
num_samples = 10000

10000

In [2]:
# --- Imports

using BenchmarkTools
using LinearAlgebra
using Statistics

In [3]:
# --- Functions

"""
    Compute QR factorization of `A` using the modified Gram-Schmidt algorithm.
"""
function mgs(A::Matrix)
    # Initialize Q and R
    Q = copy(A)
    R = zero(A)
    
    for j in 1:size(Q, 2)
        norm = LinearAlgebra.norm(Q[:, j])
        if norm > 0
            Q[:, j] /= norm
            R[j, j] = norm
        else
            continue
        end

        column = Q[:, j]
        for j_inner in (j + 1):size(Q, 2)
            R[j, j_inner] = Q[:, j_inner] ⋅ column
            Q[:, j_inner] -= R[j, j_inner] * column
        end
    end
    
    Q, R
end;

In [4]:
# --- Test mgs()

n = 10
A = randn(n, n)
Q, R = mgs(A)

println("Relative error Q * R: ", opnorm(A - Q * R)/opnorm(A))
println("Absolute error Q * R: ", opnorm(A - Q * R))
println("opnorm(Q' * Q): ", opnorm(transpose(Q) * Q))

det_A = det(A)
det_Q = det(Q)
det_R = det(R)
println("det(A): ", det_A)
println("det(Q): ", det_Q)
println("det(R): ", det_R)
println("Relative error det(Q * R):", (det_A - det_Q*det_R) / det_A)
println("Absolute error det(Q * R):", det_A - det_Q*det_R)

Relative error Q * R: 1.5512172131292414e-16
Absolute error Q * R: 7.893109130154954e-16
opnorm(Q' * Q): 1.0000000000000004
det(A): -105.74964923539657
det(Q): -1.0000000000000002
det(R): 105.74964923539655
Relative error det(Q * R):-0.0
Absolute error det(Q * R):0.0


### Run Experiments

In [5]:
# Initialize data vectors
mgs_orthogonality_errors = zeros(num_samples)
householder_orthogonality_errors = zeros(num_samples)

mgs_det_errors = zeros(num_samples)
householder_det_errors = zeros(num_samples)

# Collect data
for i in 1:num_samples
    # Generate random matrix
    A = randn(n, n)
    det_A = det(A)

    # Compute QR factorization using modified Gram-Schmidt algorithm
    Q_mgs, R_mgs = mgs(A)

    mgs_orthogonality_errors[i] = opnorm(transpose(Q_mgs) * Q_mgs - LinearAlgebra.I)

    det_Q = det(Q_mgs)
    det_R = det(R_mgs)
    mgs_det_errors[i] = abs((det_A - det_Q*det_R) / det_A)
    
    # Compute QR factorization using Householder orthogonalization
    F_householder= qr(A)
    Q_householder = F_householder.Q
    R_householder = F_householder.R
    
    householder_orthogonality_errors[i] =
        opnorm(transpose(Q_householder) * Q_householder - LinearAlgebra.I)

    det_Q = det(Q_householder)
    det_R = det(R_householder)
    householder_det_errors[i] = abs((det_A - det_Q*det_R) / det_A)
end

### Results

In [6]:
# --- Orthogonality Error

println("mean(mgs_orthogonality_errors): ", mean(mgs_orthogonality_errors))
println("std(mgs_orthogonality_errors): ", std(mgs_orthogonality_errors))

println("mean(householder_orthogonality_errors): ", mean(householder_orthogonality_errors))
println("std(householder_orthogonality_errors): ", std(householder_orthogonality_errors))

mean(mgs_orthogonality_errors): 1.3412426147818091e-14
std(mgs_orthogonality_errors): 1.952885182137655e-13
mean(householder_orthogonality_errors): 1.1494784356870544e-15
std(householder_orthogonality_errors): 2.867309089479007e-16


In [7]:
# --- Determinant Error

println("mean(mgs_det_errors): ", mean(mgs_det_errors))
println("std(mgs_det_errors): ", std(mgs_det_errors))

println("mean(householder_det_errors): ", mean(householder_det_errors))
println("std(householder_det_errors): ", std(householder_det_errors))

mean(mgs_det_errors): 3.4296252431458642e-15
std(mgs_det_errors): 5.5391668143826156e-14
mean(householder_det_errors): 5.761934017739607e-15
std(householder_det_errors): 7.265482161796808e-14
